## Dependencias

In [12]:
"""Importamos las librerias que ocuparemos mas adelante"""
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf

from datetime import date,time,datetime
from dateutil.relativedelta import relativedelta as rd

"""Relativo a conexiones con sql"""
from sqlalchemy.engine import create_engine
from sqlalchemy.types import BIGINT,INT,VARCHAR,CHAR,BLOB,DATE,DATETIME,NUMERIC,FLOAT,TEXT,BOOLEAN

import json 
import glob 
import os


cf.go_offline()

## Rutas de archivos

In [7]:
"""Rutas de archivos a las distintas tablas a limpiar"""

arch_usuario = '../Modulo5/Data/sd254_users.csv'
arch_tarj = '../Modulo5/Data/sd254_cards.csv'
arch_mcc = '../Modulo5/Data/MerchantCategoryCodeList.xls'
arch_txn = '../Modulo5/Data/credit_card_transactions-ibm_v2.csv'
arch_estados = '../Modulo5/Data/lista_estados.xlsx'


## Usuarios

In [8]:
usuarios = pd.read_csv(arch_usuario)

In [9]:
usuarios.tail(2)

,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
1998,Juelz Roman,66,60,1954,2,Male,259 Valley Boulevard,NaN,Camp Hill,PA,17011,40.24,-76.92,$25336,$54654,$27241,618,1
1999,Kenia Harris,21,60,1998,11,Female,472 Ocean View Street,NaN,Merrimack,NH,3054,42.86,-71.48,$32325,$65909,$181261,673,2


In [ ]:
cols = ['Birth Year','Birth Month','Gender','State','Zipcode','Latitude','Longitude']
#Se seleccionarón las columnas más importantes para el análisis

In [ ]:
usuarios = usuarios[cols]
#Nos quedamos solo con las columnas previamente seleccionamos

In [ ]:
usuarios.insert(0,'id_user',usuarios.index)
#Creamos una nueva columna con el indice de los usuarios, esto para usarla como llave en la base de datos

In [ ]:
usuarios['dob'] = np.vectorize(date)(usuarios['Birth Year'],usuarios['Birth Month'],1)
#Se hace un vector de fecha donde el vector esta conformado por (birth year, birth mont, 1), ya en formato date

In [ ]:
usuarios['Zipcode'] = usuarios['Zipcode'].fillna(99999).map(int).map(lambda x:'%05d'%x)
#Se llena los datos nulos con el codigo 99999 y se mapea los codigos para que solo tengan 5 digitos

In [ ]:
usuarios['Gender'] = usuarios['Gender'].map(lambda x:x[0])
#Nos quedamos con la primer letra de la columna de genero (de cada elemento)

In [ ]:
usuarios.drop(['Birth Year','Birth Month'],axis=1,inplace=True)
#Eliminamos las columnas de bith year y birth month porque ya no se ocuparan, dado que esa info ya la tenemos en dob

In [ ]:
usuarios.columns = ['id_user','gender','state','zip','lat','lon','dob']
#Renombramos las columnas de la tabla para darles un formato más limpio

In [ ]:
usuarios.head()
#Mostramos los primeros 5 elementos de la tabla

,id_user,gender,state,zip,lat,lon,dob
0,0,F,CA,91750,34.15,-117.76,1966-11-01
1,1,F,NY,11363,40.76,-73.74,1966-12-01
2,2,F,CA,91792,34.02,-117.89,1938-11-01
3,3,F,NY,10069,40.71,-73.99,1957-01-01
4,4,M,CA,94117,37.76,-122.44,1976-09-01


In [15]:
creds = json.load(open('../Modulo5//Data/creds.json','rb'))
#abrimos las credenciales para acceder a la base de daros

In [ ]:
cnx = create_engine(f'mysql+pymysql://{creds["u"]}:{creds["p"]}@{creds["h"]}/{creds["d"]}').connect()
cnx.closed
#se hace la conexion con mysql

False

In [ ]:
usuarios.to_sql(con=cnx,
name='tbl_user',
if_exists='replace',
index=False,
dtype=dict(zip(usuarios.columns,[INT,CHAR(1),CHAR(2),CHAR(5),NUMERIC(10,2),NUMERIC(10,2),DATE])))
#Cargamos la tabla en la base de datos en mysql, con tipos de datos se la sintaxis de sql

2000

## Tarjetas

In [10]:
tarj = pd.read_csv(arch_tarj)
#Leemos la tabala de tarjetas

In [ ]:
cols = ['User','CARD INDEX','Card Brand','Card Type','Card Number','Acct Open Date']
tarj = tarj[cols]
#Se seleccionas las variables mas importantes para el analisis

In [ ]:
tarj['Card Brand'] = tarj['Card Brand'].map(lambda x:x[0])
#Nos quedamos con la primer letra del card brand

In [ ]:
tarj['Card Type'] = np.where(tarj['Card Type']=='Debit (Prepaid)','P',tarj['Card Type'].map(lambda x:x[0]))
#Llenamos la columna con un condicional donde si el valor es debit prepaid se pone p y si no es asi se escoge la primer letra del valor que tome la columna

In [ ]:
tarj['Card Number'] = tarj['Card Number'].map(str)
#se hace string a los valores de la columna

In [ ]:
tarj['Acct Open Date'] = pd.to_datetime(tarj['Acct Open Date'],format='%m/%Y').dt.date
#Pasamos la columna a formato datetime

In [ ]:
tarj.head()
#mostramos los primeros elementos de la tabla

,id_user,card_index,card_brand,card_type,pan,acct_open_date
0,0,0,V,D,4344676511950444,2002-09-01
1,0,1,V,D,4956965974959986,2014-04-01
2,0,2,V,D,4582313478255491,2003-07-01
3,0,3,V,C,4879494103069057,2003-01-01
4,0,4,M,P,5722874738736011,2008-09-01


In [ ]:
tarj.columns = ['id_user','card_index','card_brand','card_type','pan','acct_open_date']
#renombramos las columnas

In [ ]:
tarj.to_sql(con=cnx,
name='tbl_card',
if_exists='replace',
index=False,
dtype=dict(zip(tarj.columns,[INT,INT,CHAR(1),CHAR(1),CHAR(16),DATE])))
#cargamos la información  a la base de datos de mysql.

6146

## MCC

In [ ]:
mcc = pd.read_excel(arch_mcc)
#Cargamos la tabla de mcc

In [ ]:
mcc.drop('Unnamed: 2',axis=1,inplace=True)
#Borramos una columna vacia que nos arroja el excel

In [ ]:
mcc.columns = ['mcc','merchant_category']
#Seleccionamos las columnas a usar

In [ ]:
mcc['mcc'] = mcc['mcc'].map(lambda x:'%04d'%x)
#Sobre la columna de mcc, hacemos que todos los codigos tengan 4 digitos

In [ ]:
mcc['merchant_category'].map(len).max()
#Checamos la lingitud de la cadena de texto mas grande de la categroría merchant category

160

In [ ]:
mcc.to_sql(con=cnx,
name='tbl_mcc',
if_exists='replace',
index=False,
dtype=dict(zip(mcc.columns,[CHAR(4),VARCHAR(160)])))
#Pasamos la tabla ya limpia a la base de datos

880

## Transacciones

In [ ]:
cols = ['Merchant State','Zip','MCC']
merchant = pd.read_csv(arch_txn,usecols=cols,dtype=str).drop_duplicates().reset_index(drop=True)
#Seleccionamos las columnas mas importante de la tabla de mechant

In [ ]:
estados = pd.read_excel(arch_estados,usecols=['state_name','state_code'])
estados.head()
#Cargamos la tabla artesanal de estados de usa, recordando que la limpieza se hizo manual

,state_name,state_code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [ ]:
merchant['territory'] = np.where(merchant['Merchant State'].isin(estados['state_code']),merchant['Merchant State'],'XX')
#Aplicamos una condicion, donde si el estado estan en la lista de estados con codigo, se deja el codigo, si no se le pone XX
merchant['MCC'] = merchant['MCC'].fillna(9999).map(int).map(lambda x:'%04d'%x)
#Para los MCC nulos se llena con el codigo 9999, se hace entero y solo van a tener 4 digitos
merchant['Zip'] = merchant['Zip'].fillna(99999).map(float).map(int).map(lambda x:'%05d'%x)
#Para el Zip los nulos se llenan con el codigo 99999, se hacen primero flotantes, luego entero y se le pone que la columna este llena de 5 digitos

In [ ]:
merchant = merchant[~((merchant['Zip']=='99999')|(merchant['MCC']=='9999'))]
#Guardamos aquellos datos que no cumplan con tener codigo zip 99999 y mc 9999
#(merchant['Zip']!='99999')&(merchant['MCC']!='9999')

In [ ]:
merchant.drop('Merchant State',axis=1,inplace=True)
#Eliminamos la columna de Merchant State

In [ ]:
merchant = merchant.merge(mcc,left_on='MCC',right_on='mcc',how='inner')
#Unimos las tablas de merchant y mcc usando como llave "MCC" y "mcc", con un inner join (la inteseccion de los 2 conjuntos)

In [ ]:
merchant.drop(['mcc','merchant_category'],axis=1,inplace=True)
#De la tabla merchant eliminamos las columnas mcc y merchant_category

In [ ]:
merchant.insert(0,'id',merchant.index+1)
#Creamos una nueva tabla con el id de merchant

In [ ]:
merchant.head()
#Visualizamos los primeros 5 elementos de la tabla

,id,Zip,MCC,territory
0,1,91750,5300,CA
1,2,79927,5300,TX
2,3,60643,5300,IL
3,4,48170,5300,MI
4,5,98198,5300,WA


In [ ]:
merchant.columns =['id_merchant','zip_merchant','mcc','territory']
#Renombramos las variables de la tabla

In [ ]:
merchant.head()

,id_merchant,zip_merchant,mcc,territory
0,1,91750,5300,CA
1,2,79927,5300,TX
2,3,60643,5300,IL
3,4,48170,5300,MI
4,5,98198,5300,WA


In [ ]:
merchant.to_sql(con=cnx,
name='tbl_merchant',
if_exists='replace',
index=False,
dtype=dict(zip(merchant.columns,[INT,CHAR(5),CHAR(4),CHAR(2)])))
#cargamos la tabla de merchant a la base de datos

248870

In [ ]:
cols = ['User','Card','Year','Month','Day','Time','Amount','Use Chip','Merchant State','Zip','MCC','Errors?','Is Fraud?']
txn = pd.read_csv(arch_txn,usecols=cols,dtype=str)
#Cargamos la tabla de transacciones, con nrows podemos hacer una muestra pequeña para hacer la limpieza mas rapido y luego hacerlo de manera general, seleccionamos las columnas
#mas importantes segun nuestro criterio

In [ ]:
txn['date'] = np.vectorize(date)(txn['Year'].astype(int),txn['Month'].astype(int),txn['Day'].astype(int))
#Hacemos una vectorizacion de variable date, donde lo llenamos con Year con formato de enter, month con formato de entero y day con formato de entero

In [ ]:
txn['ts'] = pd.to_datetime(txn['date'].map(lambda x:x.strftime('%Y-%m-%d'))+' '+txn['Time'],format='%Y-%m-%d %H:%M')
#Creamos un timestamp donde juntamos las columnas de date y time

In [ ]:
txn.drop(['Year','Month','Day','Time','date'],axis=1,inplace=True)
#Eliminamos las columnas que ya no usaran por la creacion de la variable ts

In [ ]:
txn['Amount'] = txn['Amount'].map(lambda x:x.replace('$','')).map(float)
#De la columna amount quitamos el signo de pesos y lo cambiamos por vacio, ademas lo convertimos en flotante

In [ ]:
txn['Is Fraud?'] = (txn['Is Fraud?']=='Yes').astype(int)
#La variable Is fraud lo convertimos en entero donde yes es 1 y no es 0

In [ ]:
txn['territory'] = np.where(txn['Merchant State'].isin(estados['state_code']),txn['Merchant State'],'XX')
txn['MCC'] = txn['MCC'].fillna(9999).map(int).map(lambda x:'%04d'%x)
txn['Zip'] = txn['Zip'].fillna(99999).map(float).map(int).map(lambda x:'%05d'%x)
#Aplicamos las mismas tranformaciones hechas a la variable merchant

In [ ]:
merchant

,id_merchant,zip_merchant,mcc,territory
0,1,91750,5300,CA
1,2,79927,5300,TX
2,3,60643,5300,IL
3,4,48170,5300,MI
4,5,98198,5300,WA
...,...,...,...,...
248865,248866,92553,5192,CA
248866,248867,76262,5192,TX
248867,248868,33183,5192,FL
248868,248869,37918,5192,TN


In [ ]:
txn.rename(columns={'Zip':'zip_merchant','MCC':'mcc'},inplace=True)
#Renombramos columnas

In [ ]:
txn = txn.merge(merchant,
on=['zip_merchant','mcc','territory'],
how='inner').drop(['zip_merchant','mcc','territory','Merchant State'],axis=1)
#Juntamos la tabla de transacciones con merchan y borramos las variables repetids

In [ ]:
txn.head(5)

,User,Card,Amount,Use Chip,Errors?,Is Fraud?,ts,id_merchant
0,0,0,134.09,Swipe Transaction,NaN,0,2002-09-01 06:21:00,1
1,0,0,102.18,Swipe Transaction,NaN,0,2002-09-10 06:22:00,1
2,0,0,126.79,Swipe Transaction,NaN,0,2002-09-13 06:22:00,1
3,0,0,115.34,Swipe Transaction,NaN,0,2002-09-16 06:00:00,1
4,0,0,128.85,Swipe Transaction,NaN,0,2002-09-18 06:19:00,1


In [ ]:
txn[['User','Card']] = txn[['User','Card']].astype(int)
#Las variables user y cars se hacen numerixas

In [ ]:
txn = txn.merge(tarj[['id_user','card_index','pan']],left_on=['User','Card'],right_on=['id_user','card_index'],how='inner')
#Junramos la tabla de transacciones con las de tarjetas tomando solo unas cuantas columnas, y se toma como llave a user y card

In [ ]:
txn.drop(['Card','card_index','User'],axis=1,inplace=True)
#Eliminamos las columnas que no nos puedan servir, esto para conservar el pan

In [ ]:
txn['Use Chip'] = txn['Use Chip'].map(lambda x:x[0])
#Nos quedamos con la primer letra del use chip

In [ ]:
txn.insert(0,'id_txn',txn.index+1) #creamos una nueva columna con el indice +1
txn.columns = ['id_txn','amount','txn_type','rej_reason','is_fraud','ts','id_merchant','id_user','pan'] #nombramos las columnas

In [ ]:
txn['rej_reason'].fillna('').map(len).max()
#Para la variable rej_reason llenamos los nulos con vacio vemos su longitud maxima para el tipo de dato para sql

37

In [ ]:
txn.head()

,id_txn,amount,txn_type,rej_reason,is_fraud,ts,id_merchant,id_user,pan
0,1,134.09,S,NaN,0,2002-09-01 06:21:00,1,0,4344676511950444
1,2,102.18,S,NaN,0,2002-09-10 06:22:00,1,0,4344676511950444
2,3,126.79,S,NaN,0,2002-09-13 06:22:00,1,0,4344676511950444
3,4,115.34,S,NaN,0,2002-09-16 06:00:00,1,0,4344676511950444
4,5,128.85,S,NaN,0,2002-09-18 06:19:00,1,0,4344676511950444


In [ ]:
txn.to_sql(con=cnx,chunksize=1000000,
name='tbl_txn',
if_exists='replace',
index=False,
dtype=dict(zip(txn.columns,[INT,NUMERIC(10,2),CHAR(1),VARCHAR(37),BOOLEAN,DATETIME,INT,INT,CHAR(16)])))
#Cargamos los datos en la base de datos de mysql

21508765

In [ ]:
txn['ts'].map(lambda x:x.strftime('%Y%m%d')).nunique()
#Checamos cuantos dias distintos hay con transacciones registradas

10596